In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
from matplotlib import rcParams
import scanpy as sc
import os
import anndata
from sklearn.mixture import GaussianMixture
from fcsy import DataFrame
import matplotlib
from glob import glob
matplotlib.rcParams['pdf.fonttype']=42
matplotlib.rcParams['ps.fonttype']=42
import warnings
warnings.filterwarnings("ignore")
from igraph import InternalError

# scanpy settings
sc.settings.verbosity = 0  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.set_figure_params(dpi=150, frameon=False, figsize=(4, 4)) 
sc._settings.ScanpyConfig.n_jobs=4 # useless

In [2]:
# readin the information table
info_path = '/Users/tan/ionctura-collab/data/Sample_info_reformat.xlsx'
sampleInfo = pd.read_excel(info_path, dtype={'Sample ID':str}, sheet_name=0)

In [3]:
sampleInfo

,Tumor_Groups,Subject_ID,Tumor_type,Cohort_and_dose,timepoint,Sampling date,Gender,Sample_ID,Sample nr in a 96 well plate,Barcode key,Barcoded Batch,Expt_ID
0,Tumor Group 1,3901-01,Melanoma / Follicular NHL (Stage IV),1 (10mg),AC2D1,2020-03-25,M,100094,1,1,1,EXP-21-DG3637
1,Tumor Group 1,3901-01,Melanoma / Follicular NHL (Stage IV),1 (10mg),AC3D1,2020-04-20,M,100095,2,2,1,EXP-21-DG3637
2,Tumor Group 1,3901-01,Melanoma / Follicular NHL (Stage IV),1 (10mg),AC4D1,2020-05-18,M,100096,3,3,1,EXP-21-DG3637
3,Tumor Group 1,3901-01,Melanoma / Follicular NHL (Stage IV),1 (10mg),AC5D1,2020-06-16,M,100097,4,5,1,EXP-21-DG3637
4,Tumor Group 1,3901-01,Melanoma / Follicular NHL (Stage IV),1 (10mg),AC6D1,2020-07-14,M,100098,5,6,1,EXP-21-DG3637
...,...,...,...,...,...,...,...,...,...,...,...,...
112,Tumor Group 3,4401-06,Uveal Melanoma (Stage IV),3 (40mg),AC1D1,2020-11-10,M,100158,45,6,5,EXP-21-DG3637
113,Tumor Group 3,4401-06,Uveal Melanoma (Stage IV),3 (40mg),AC1D2,2020-11-11,M,100159,46,8,5,EXP-21-DG3637
114,Tumor Group 3,4401-06,Uveal Melanoma (Stage IV),3 (40mg),AC1D15,2020-11-24,M,100160,47,11,5,EXP-21-DG3637
115,Tumor Group 3,4401-06,Uveal Melanoma (Stage IV),3 (40mg),AC2D1,2020-12-08,M,100161,48,12,5,EXP-21-DG3637


In [4]:
selectInfo = sampleInfo

In [5]:
#check panel
i1=116
i2=0
dataDir = '/Users/tan/cytof_data/*/renamed'
data_path1 = glob(dataDir + '/*/' + str(selectInfo['Sample_ID'].iloc[i1]) + '*.fcs')[0]
panel1 = DataFrame.from_fcs(data_path1, channel_type='long').columns.sort_values()
data_path2 = glob(dataDir + '/*/' + str(selectInfo['Sample_ID'].iloc[i2]) + '*.fcs')[0]
panel2 = DataFrame.from_fcs(data_path2, channel_type='long').columns.sort_values()

In [6]:
panel2.intersection(panel1)

Index(['102Pd', '104Pd', '105Pd', '106Pd', '108Pd', '116Cd', '131Xe', '191Ir',
       '193Ir', 'CD11c', 'CD123', 'CD127', 'CD137', 'CD14', 'CD141', 'CD147',
       'CD16', 'CD161', 'CD1c', 'CD20', 'CD22', 'CD24', 'CD25', 'CD26',
       'CD27 ', 'CD28', 'CD29', 'CD3', 'CD33', 'CD38', 'CD39', 'CD4', 'CD43',
       'CD45', 'CD45RA', 'CD49d', 'CD5', 'CD52', 'CD55', 'CD56', 'CD57',
       'CD64', 'CD7', 'CD71', 'CD81', 'CD85j', 'CD8a', 'CD9', 'CD95', 'CD99',
       'CX3CR1', 'Center', 'Event_length', 'HLA-DR', 'IgD', 'Offset',
       'Residual', 'Siglec-8', 'TCRgd', 'Time', 'Width', 'pan-KIR'],
      dtype='object')

In [7]:
dataDir = '/Users/tan/cytof_data/*/renamed'
labelDir = '/Users/tan/cytof_data/*/classifiedSlim'
nsample = 3000

select_columns = ['CD11c', 'CD123', 'CD127', 'CD137', 'CD14', 'CD141', 'CD147',
       'CD16', 'CD161', 'CD1c', 'CD20', 'CD22', 'CD24', 'CD25', 'CD26',
       'CD27 ', 'CD28', 'CD29', 'CD3', 'CD33', 'CD38', 'CD39', 'CD4', 'CD43',
       'CD45', 'CD45RA', 'CD49d', 'CD5', 'CD52', 'CD55', 'CD56', 'CD57',
       'CD64', 'CD7', 'CD71', 'CD81', 'CD85j', 'CD8a', 'CD9', 'CD95', 'CD99',
       'CX3CR1', 'HLA-DR', 'IgD', 'Siglec-8', 'TCRgd', 'pan-KIR']
drop_dic = {'pDC': [], 
            'B-cells': ['CD33', 'CD3', 'TCRgd', 'Siglec-8', 'CD14', 'CD141', 'CD4', 'pan-KIR'],
            'CD4 T-cells': ['IgD', 'CD1c', 'TCRgd', 'Siglec-8', 'CD20', 'CD14'],
            'CD8 T-cells': ['IgD', 'CD11c', 'CD1c', 'TCRgd', 'Siglec-8', 'CD20', 'CD14'],
            #'Eosinophils': ['IgD', 'CD57', 'CD25', 'TCRgd', 'CD14', 'pan-KIR'],
            'Monocytes': ['CD57', 'IgD', 'CD25', 'CD20', 'TCRgd', 'CD22', 'CD127', 'pan-KIR'], 
            'Neutrophils': ['IgD', 'HLA-DR', 'CD57', 'CD25', 'CD22', 'TCRgd', 'CD123', 'CD161', 'pan-KIR'],
            'NK cells': [],
            'Lin Neg': [],
            #'gdT': [], 
            'Plasmablasts': [],
            'Basophils': [],
            #'Tregs':[]
           }

# readin and merge the file according to "selectInfo" as an anndata object
all_data_path = '../adata/ion_EXP-21-DG3637_56_Slim.h5ad'

#for sub_name in drop_dic:
#print('currently working on ' + sub_name)
if not os.path.exists(all_data_path):
    print('adata not found, load and preprocess raw data...')
    all_data_list = []
    all_label_list = []
    for i in range(len(selectInfo)):
        label_path = glob(labelDir + '/*/' + str(selectInfo['Sample_ID'].iloc[i]) + '*.csv')[0]
        data_path = glob(dataDir + '/*/' + str(selectInfo['Sample_ID'].iloc[i]) + '*.fcs')[0]
        labelTmp = pd.read_csv(label_path)
        labelTmp['Sample ID'] = np.repeat(str(selectInfo['Sample_ID'].iloc[i]), len(labelTmp.index))
        labelTmp['timepoint'] = np.repeat(str(selectInfo['timepoint'].iloc[i]), len(labelTmp.index))
        labelTmp['Subject ID'] = np.repeat(str(selectInfo['Subject_ID'].iloc[i]), len(labelTmp.index))
        labelTmp['dose'] = np.repeat(str(selectInfo['Cohort_and_dose'].iloc[i]), len(labelTmp.index))
        labelTmp['type'] = np.repeat(str(selectInfo['Tumor_type'].iloc[i]), len(labelTmp.index))
        labelTmp['tumor_group'] = np.repeat(str(selectInfo['Tumor_Groups'].iloc[i]), len(labelTmp.index))
        labelTmp['timepoint_dose'] = np.repeat(selectInfo['timepoint'].iloc[i] + 
                                                '_' + 
                                                str(selectInfo['Cohort_and_dose'].iloc[i]), len(labelTmp.index))
        labelTmp['batch'] = np.repeat(data_path.split('/')[-4], len(labelTmp.index)) #EXP-XX-XXXXXX
        dataTmp = DataFrame.from_fcs(data_path, channel_type='long')
        if '4-1BB' in set(dataTmp.columns):
            dataTmp.rename(columns={"4-1BB": "CD137"}, inplace=True)
        # filter the cells without a level1 tag
        dataTmp = dataTmp[labelTmp['level1']!=' ']
        labelTmp = labelTmp[labelTmp['level1']!=' ']
        # remove EQBeads and DNA channel # also remove the negative channels
        #dataTmp.drop(columns=drop_columns, inplace=True)
        dataTmp = dataTmp[select_columns]
        dataTmp = np.arcsinh(dataTmp/5)
        all_data_list.append(dataTmp)
        all_label_list.append(labelTmp)

    all_data = pd.concat(all_data_list, ignore_index=True)
    all_label = pd.concat(all_label_list, ignore_index=True)
    adata = anndata.AnnData(all_data)
    adata.obs = all_label
    print('batch correction...')
    sc.pp.combat(adata, key = 'batch', covariates = ['tumor_group', 'timepoint', 'dose'])
    print('scaling...')
    sc.pp.scale(adata)
    os.makedirs('adata/', exist_ok=True)
    print('write to h5ad file...')
    adata.write(filename=all_data_path, compression = 'gzip')
    adata=None
    print('finished!')
print('loading...')
adata_all = sc.read_h5ad(filename = all_data_path)
print('finished!')

adata not found, load and preprocess raw data...
batch correction...


... storing 'level0' as categorical
... storing 'level1' as categorical
... storing 'level2' as categorical
... storing 'Sample ID' as categorical
... storing 'timepoint' as categorical
... storing 'Subject ID' as categorical
... storing 'dose' as categorical
... storing 'type' as categorical
... storing 'tumor_group' as categorical
... storing 'timepoint_dose' as categorical
... storing 'batch' as categorical


scaling...
write to h5ad file...
finished!
loading...
finished!


In [ ]:
adata_all.obs['Sample ID'].nunique()

In [ ]:
adata_all.obs

In [46]:
pd.set_option('display.max_rows', adata.obs['timepoint'].nunique()+1)
adata.obs['timepoint'].value_counts()[adata.obs['timepoint'].value_counts()>20].index.sort_values().tolist()

['AC1D1',
 'AC1D2',
 'AC1D15',
 'AC2D1',
 'AC3D1',
 'AC4D1',
 'AC5D1',
 'AC9D1',
 'AC10D1',
 'AC11D1',
 'AC12D1',
 'AC13D1',
 'AC14D1',
 'AC15D1',
 'AC17D1',
 'AC18D1']

In [ ]:
density_groups = adata.obs['timepoint'].cat.categories.tolist()

In [ ]:
adata.obs['timepoint'].cat.categories.tolist()

In [62]:
tumor_group='Tumor Group 2'
for sub_name in drop_dic:
    # subsampling
    print('subsampling...')
    try:
        if sub_name in adata_all.obs['level1'].unique().tolist():
            sample_index = adata_all.obs[adata_all.obs['level1']==sub_name].groupby('Sample ID').apply(lambda x: x.sample(n=nsample, random_state=0) if x.shape[0]>=nsample else x).index.droplevel(level=0)
        elif sub_name in adata_all.obs['level2'].unique().tolist():
            sample_index = adata_all.obs[adata_all.obs['level2']==sub_name].groupby('Sample ID').apply(lambda x: x.sample(n=nsample, random_state=0) if x.shape[0]>=nsample else x).index.droplevel(level=0)
        elif sub_name == 'Tregs':
            sample_index = adata_all.obs[adata_all.obs['level3'].isin(['CD39 Memory Tregs', 'Memory Tregs', 'Naive Tregs'])].groupby('Sample ID').apply(lambda x: x.sample(n=nsample, random_state=0) if x.shape[0]>=nsample else x).index.droplevel(level=0)        
        adata_sample = adata_all[sample_index]
    except ValueError as e: 
        # usually when no file has more cells than nsample and thus no subsampling at all.
        # so the results after apply will not be a multiplex index and the droplevel func will fail.
        print(e)
        if sub_name in adata_all.obs['level1'].unique().tolist():
            adata_sample = adata_all[adata_all.obs['level1']==sub_name]
        elif sub_name in adata_all.obs['level2'].unique().tolist():
            adata_sample = adata_all[adata_all.obs['level2']==sub_name]
        elif sub_name == 'Tregs':
            adata_sample = adata_all[adata_all.obs['level3'].isin(['CD39 Memory Tregs', 'Memory Tregs', 'Naive Tregs'])]

    adata = anndata.AnnData(adata_sample.to_df().drop(columns = drop_dic[sub_name]))
    adata.obs = pd.DataFrame(adata_sample.obs)
    
    adata = adata[adata.obs['tumor_group'] == tumor_group] # merge by tumor group
    
    # filter out those density_groups that have too few cells
    density_groups = adata.obs['timepoint'].value_counts()[adata.obs['timepoint'].value_counts()>20].index.sort_values().tolist()
    adata = adata[adata.obs['timepoint'].isin(density_groups)]
    # skip
    # optimization
    #sc.settings.figdir='./optimization/'
    #for res in [0.3]:
    #    adata_opt = adata
    #    n_comps = min([adata_opt.n_obs, adata_opt.n_vars, 21])-1
    #    sc.tl.pca(adata_opt, svd_solver='arpack', n_comps=n_comps)
    #    sc.pp.neighbors(adata_opt, n_neighbors=10, n_pcs=n_comps)
    #    sc.tl.leiden(adata_opt, resolution=res)
    #    sc.tl.paga(adata_opt, groups='leiden')
    #    sc.pl.paga(adata_opt, color=['leiden'], threshold=0.1, show=False, 
    #               save='_' + sub_name + '_' + str(res) + '.pdf')
    #    adata_opt = []

    # save figures to a sub dir
    figpath='../figures/' + tumor_group + '/' + sub_name + '/'
    os.makedirs(figpath, exist_ok=True)
    sc.settings.figdir=figpath

    print('calculating PAGA...')
    #n_comps = min([adata.n_obs, adata.n_vars, 21])-1
    #sc.tl.pca(adata, svd_solver='arpack', n_comps=n_comps)
    sc.pp.neighbors(adata, n_neighbors=10)

    # paga process
    sc.tl.leiden(adata, resolution=0.3) 

    sc.tl.paga(adata, groups='leiden')
    if adata.obs['leiden'].nunique() < 2: # no paga can be done with only 1 group
        sc.tl.draw_graph(adata)
        sc.pl.draw_graph(adata, color=['timepoint', 'dose', 'leiden', 'batch', 'Subject ID', 'type'], show=False,
                     save='_' + sub_name + '.pdf')
        continue
    try:
        sc.pl.paga(adata, color=['leiden'], threshold=0.1, show=False, 
                   save='_' + sub_name + '.pdf')
    except InternalError as e: # maybe there're too little cells
        print(e)
        sc.pl.paga(adata, color=['leiden'], show=False, 
                   save='_' + sub_name + '.pdf')        

    print('embedding with FA...')
    sc.tl.draw_graph(adata, init_pos='paga')

    sc.pl.draw_graph(adata, color=['timepoint', 'dose', 'leiden', 'batch', 'Subject ID', 'type'], show=False,
                     save='_' + sub_name + '.pdf')

    sc.pl.draw_graph(adata, color=adata.var.index.values, show=False,
                     save='_' + sub_name + '_markers.pdf')

    print('embedding with density plot...')
    

    sc.tl.embedding_density(adata, basis='draw_graph_fa', groupby='timepoint')
    sc.pl.embedding_density(adata, basis='draw_graph_fa', key='draw_graph_fa_density_timepoint', 
                            group=density_groups, show=False, 
                            save='_' + sub_name + '_timepoint_density.pdf')

    print('saving results...')
    os.makedirs('../PAGA_result_data/', exist_ok=True)
    adata.write(filename='../PAGA_result_data/EXP-21-DG3637_56_' + sub_name + '_sample3000.h5ad', compression = 'gzip')
    print('done!')

subsampling...
Cannot remove 1 levels from an index with 1 levels: at least one level must be left.
calculating PAGA...
embedding with FA...
embedding with density plot...
saving results...
done!
subsampling...
calculating PAGA...
embedding with FA...
embedding with density plot...
saving results...
done!
subsampling...
calculating PAGA...
embedding with FA...
embedding with density plot...
saving results...
done!
subsampling...
calculating PAGA...
subsampling...
calculating PAGA...
embedding with FA...
embedding with density plot...
saving results...
done!
subsampling...
calculating PAGA...
embedding with FA...
embedding with density plot...
saving results...
done!
subsampling...
calculating PAGA...
embedding with FA...
embedding with density plot...
saving results...
done!
subsampling...
calculating PAGA...
embedding with FA...
embedding with density plot...
saving results...
done!
subsampling...
Cannot remove 1 levels from an index with 1 levels: at least one level must be left.
cal

In [57]:
sc.tl.draw_graph(adata)
sc.pl.draw_graph(adata, color=['timepoint', 'dose', 'leiden', 'batch', 'Subject ID', 'type'], show=False,
                     save='_' + sub_name + '.pdf')

[<AxesSubplot:title={'center':'timepoint'}, xlabel='FA1', ylabel='FA2'>,
 <AxesSubplot:title={'center':'dose'}, xlabel='FA1', ylabel='FA2'>,
 <AxesSubplot:title={'center':'leiden'}, xlabel='FA1', ylabel='FA2'>,
 <AxesSubplot:title={'center':'batch'}, xlabel='FA1', ylabel='FA2'>,
 <AxesSubplot:title={'center':'Subject ID'}, xlabel='FA1', ylabel='FA2'>,
 <AxesSubplot:title={'center':'type'}, xlabel='FA1', ylabel='FA2'>]

In [19]:
adata.obs['timepoint'].cat.categories.tolist()[0:11]

['AC1D1',
 'AC1D2',
 'AC1D15',
 'AC2D1',
 'AC3D1',
 'AC4D1',
 'AC5D1',
 'AC6D1',
 'AC7D1',
 'AC8D1',
 'AC9D1']